We can easily simulate the subdivision for several steps in Python:

In [1]:
#!/usr/bin/env python3

import collections


def main():
    q = collections.deque()
    # (depth, x1, x2, y1, y2, horizontal)
    q.append((1, 0, 4096, 0, 2048, True))
    rectcount = collections.defaultdict(int)
    curr_depth = 0
    while q:
        depth, x1, x2, y1, y2, horizontal = q.popleft()
        if depth > curr_depth:
            count = sum(1 for v in rectcount.values() if v == 4)
            rectcount = collections.defaultdict(int)
            print(f"f({curr_depth}) = {count}")
            curr_depth = depth
        if depth > 10:
            break
        rectcount[(x1, y1)] += 1
        rectcount[(x1, y2)] += 1
        rectcount[(x2, y1)] += 1
        rectcount[(x2, y2)] += 1
        depth += 1
        if horizontal:
            x3 = (3 * x1 + x2) // 4
            x4 = (x1 + 3 * x2) // 4
            y3 = (y1 + y2) // 2
            q.append((depth, x1, x3, y1, y2, False))
            q.append((depth, x3, x4, y1, y3, True))
            q.append((depth, x3, x4, y3, y2, True))
            q.append((depth, x4, x2, y1, y2, False))
        else:
            x3 = (x1 + x2) // 2
            y3 = (3 * y1 + y2) // 4
            y4 = (y1 + 3 * y2) // 4
            q.append((depth, x1, x2, y1, y3, True))
            q.append((depth, x1, x3, y3, y4, False))
            q.append((depth, x3, x2, y3, y4, False))
            q.append((depth, x1, x2, y4, y2, True))


if __name__ == "__main__":
    main()


f(0) = 0
f(1) = 0
f(2) = 0
f(3) = 2
f(4) = 16
f(5) = 82
f(6) = 368
f(7) = 1554
f(8) = 6384
f(9) = 25874
f(10) = 104176


Apparently each step we quadruple the number of internal points, then add a number of points on the edges. Inspecting $g(n) = f(n+1) - 4f(n)$ we get $\{2, 8, 18, 40, 82, 168, 338, 680, \dots\}$. Hard not to notice

$$g(n) = 2g(n-1) + 3 + (-1)^{n+1},\ g(1) = 0.$$

Solving this recurrence equation with Mathematica

```mathematica
RSolve[{g[n] == 2 g[n - 1] + 3 - (-1)^n, g[1] == 0}, g[n], n]
```

gives us

$$g(n) = \frac{1}{3}(2^{n+2} + (-1)^{n+1} - 9).$$

Plugging into the recurrence equation of $f$, again with Mathematica

```mathematica
RSolve[{f[n] == 4 f[n - 1] + 1/3 (-9 + (-1)^(1 + n) + 2^(2 + n)), 
  f[1] == 0}, f[n], n]
```

we get

$$f(n) = \frac{1}{15}(3 \cdot 2^{2n+1} - 5 \cdot 2^{n+2} + (-1)^{n+1}) + 1 = \frac{1}{15}(6 \cdot (2^n)^2 - 20 \cdot 2^n - (-1)^n) + 1.$$